In [ ]:
# # === IMPORTS ===
import carla
import time
import threading
import pandas as pd
import random
import os
import sys
import numpy as np
import cv2
import datetime

# === SETUP ===
os.makedirs('output/video', exist_ok=True)
sys.path.append('D:/carla/PythonAPI')
sys.path.append('D:/carla/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
from agents.navigation.behavior_agent import BehaviorAgent

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
map = world.get_map()
blueprints = world.get_blueprint_library()
spawn_points = map.get_spawn_points()

for a in world.get_actors().filter('*vehicle*'): a.destroy()
for s in world.get_actors().filter('*sensor*'): s.destroy()

In [ ]:
# === LABEL SPAWN POINTS ===
print("\U0001F4CD Drawing enlarged label numbers at each spawn point...")
for idx, sp in enumerate(spawn_points):
    loc = sp.location + carla.Location(z=2.5)
    for dx, dy in [(-0.15,0), (0.15,0), (0,-0.15), (0,0.15), (0,0)]:
        offset_loc = loc + carla.Location(x=dx, y=dy)
        world.debug.draw_string(offset_loc, f"#{idx}", draw_shadow=True,
            color=carla.Color(r=255, g=255, b=0), life_time=100.0, persistent_lines=True)
print(f"✅ {len(spawn_points)} bold label numbers displayed in simulator.")

# === INPUT ===
start_index = int(input("\nEnter index for START location: "))
spoof_index = int(input("Enter index for SPOOFED GPS location: "))
end_index = int(input("Enter index for FINAL DESTINATION: "))

start_transform = spawn_points[start_index]
spoof_location = spawn_points[spoof_index].location
end_transform = spawn_points[end_index]

In [ ]:
# === FILE NAMING ===
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
video_fps = 20

# === COMMON FUNCTIONS ===
def follow_vehicle(vehicle):
    spectator = world.get_spectator()
    while vehicle.is_alive:
        transform = vehicle.get_transform()
        spectator.set_transform(carla.Transform(
            transform.transform(carla.Location(x=-6, z=3)),
            transform.rotation))
        time.sleep(0.05)

def run_simulation(vehicle_bp_id, spoofing_enabled=False, affect_steering=False, run_tag="normal"):
    vehicle_bp = blueprints.filter(vehicle_bp_id)[0]
    vehicle = world.try_spawn_actor(vehicle_bp, start_transform)
    print(f"\U0001F697 Vehicle spawned for {run_tag} run.")

    sensor_data = {'gnss': None, 'imu': None}
    data_records = []
    frame_id = 0
    spoofing_active = False

    video_paths = {
        'rgb': f'output/video/rgb_video_{run_tag}_{timestamp}.avi',
        'depth': f'output/video/depth_video_{run_tag}_{timestamp}.avi',
        'seg': f'output/video/segmentation_video_{run_tag}_{timestamp}.avi',
        'lidar': f'output/video/lidar_video_{run_tag}_{timestamp}.avi'
    }
    video_writers = {}

    def gps_callback(data):
        sensor_data['gnss'] = {
            'latitude': spoof_location.x if spoofing_active else data.latitude,
            'longitude': spoof_location.y if spoofing_active else data.longitude
        }

    def imu_callback(data):
        sensor_data['imu'] = {
            'accel_x': data.accelerometer.x, 'accel_y': data.accelerometer.y, 'accel_z': data.accelerometer.z,
            'gyro_x': data.gyroscope.x, 'gyro_y': data.gyroscope.y, 'gyro_z': data.gyroscope.z
        }

    def rgb_callback(image):
        nonlocal frame_id
        if not (sensor_data['gnss'] and sensor_data['imu']): return
        array = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))
        rgb_img = array[:, :, :3][:, :, ::-1]
        if 'rgb' not in video_writers:
            video_writers['rgb'] = cv2.VideoWriter(video_paths['rgb'], cv2.VideoWriter_fourcc(*'XVID'), video_fps, (image.width, image.height))
        video_writers['rgb'].write(rgb_img)
        vel = vehicle.get_velocity()
        speed = 3.6 * np.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
        control = vehicle.get_control()
        data_records.append({
            'frame_id': frame_id,
            'image_num': image.frame,
            'latitude': sensor_data['gnss']['latitude'],
            'longitude': sensor_data['gnss']['longitude'],
            'accel_x': sensor_data['imu']['accel_x'],
            'accel_y': sensor_data['imu']['accel_y'],
            'accel_z': sensor_data['imu']['accel_z'],
            'gyro_x': sensor_data['imu']['gyro_x'],
            'gyro_y': sensor_data['imu']['gyro_y'],
            'gyro_z': sensor_data['imu']['gyro_z'],
            'steering_angle': control.steer,
            'throttle': control.throttle,
            'brake': control.brake,
            'speed': speed,
            'label': 'spoofed' if spoofing_enabled else 'normal',
            'run_tag': run_tag
        })
        frame_id += 1

    # Attach Sensors
    camera_bp = blueprints.find('sensor.camera.rgb')
    gnss = world.spawn_actor(blueprints.find('sensor.other.gnss'), carla.Transform(), attach_to=vehicle)
    imu = world.spawn_actor(blueprints.find('sensor.other.imu'), carla.Transform(), attach_to=vehicle)
    camera = world.spawn_actor(camera_bp, carla.Transform(carla.Location(x=1.5, z=2.4)), attach_to=vehicle)
    gnss.listen(gps_callback)
    imu.listen(imu_callback)
    camera.listen(rgb_callback)

    sensors = [gnss, imu, camera]
    threading.Thread(target=follow_vehicle, args=(vehicle,), daemon=True).start()

    if spoofing_enabled:
        def spoofing_trigger():
            nonlocal spoofing_active
            print(f"\U0001F551 Spoofing starts in 10s...")
            time.sleep(10)
            spoofing_active = True
            print("\U0001F6A8 Spoofing activated.")
        threading.Thread(target=spoofing_trigger).start()

In [ ]:
    # Routing
    grp = GlobalRoutePlanner(map, 2.0)
    waypoints = grp.trace_route(start_transform.location, end_transform.location)
    for wp, _ in waypoints:
        loc = wp.transform.location + carla.Location(z=0.5)
        world.debug.draw_point(loc, size=0.1, color=carla.Color(0, 0, 255), life_time=100.0)

    agent = BehaviorAgent(vehicle, behavior='normal')
    agent.set_destination(end_transform.location)

    while vehicle.get_location().distance(end_transform.location) > 2.0:
        world.tick()
        control = agent.run_step()
        if spoofing_enabled and affect_steering and spoofing_active:
            drift = 0.4 * np.sin(frame_id / 30.0)
            control.steer = np.clip(control.steer + drift, -1.0, 1.0)
            control.throttle = max(0.3, control.throttle)
            control.brake = 0.0
        vehicle.apply_control(control)

In [ ]:
    # Cleanup
    for s in sensors:
        try: s.stop(); s.destroy()
        except: pass
    try: vehicle.destroy()
    except: pass
    for vw in video_writers.values():
        try: vw.release()
        except: pass

    return data_records

In [ ]:
# === RUN FIRST CAR ===
data_vehicle1 = run_simulation('vehicle.lincoln.mkz_2020', spoofing_enabled=True, affect_steering=False, run_tag="vehicle1")

In [ ]:
# === RUN SECOND CAR ===
data_vehicle2 = run_simulation('vehicle.mini.cooperst', spoofing_enabled=True, affect_steering=True, run_tag="vehicle2")

In [ ]:
# === SAVE FINAL CSV ===
try:
    pd.DataFrame(data_vehicle1).to_csv(f'combined_data_vehicle1_{timestamp}.csv', index=False)
    pd.DataFrame(data_vehicle2).to_csv(f'combined_data_vehicle2_{timestamp}.csv', index=False)
    print("\U0001F4BE Data CSVs saved for both vehicles.")
except Exception as e:
    print(f"Error saving CSV: {e}")

print("\u2705 All done. Both runs completed and saved.")
